In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import  StandardScaler

In [3]:
from sklearn.model_selection import  train_test_split

In [4]:
from mxnet import  autograd,nd

In [5]:
from mxnet.gluon import data as gdata

In [6]:
from mxnet.gluon import loss  as gloss,nn

In [7]:
from mxnet import gluon,init

In [8]:
import gluonbook as gb

In [35]:
train_data = pd.read_csv("Processed_train.csv")
commit = pd.read_csv("Processed_test.csv")
X_train_data = train_data.iloc[:,1:]
y_train_data = train_data.iloc[:,0]
scaler = StandardScaler()
scaler.fit(X_train_data)
X_train_data = scaler.transform(X_train_data)
commit = scaler.transform(commit)
print(X_train_data)

[[ 0.82737724  0.73769513 -0.42438276 ...,  0.05915988 -1.2316449
  -0.7075504 ]
 [-1.56610693 -1.35557354  0.7684407  ...,  0.05915988 -1.2316449
   1.2352151 ]
 [ 0.82737724 -1.35557354 -0.42438276 ..., -0.56097483  0.81192233
   0.26383235]
 ..., 
 [ 0.82737724 -1.35557354 -0.42438276 ...,  1.29942929 -1.2316449
   0.26383235]
 [-1.56610693  0.73769513 -0.42438276 ..., -0.56097483  0.81192233
  -0.7075504 ]
 [ 0.82737724  0.73769513 -0.42438276 ..., -0.56097483  0.81192233
  -0.7075504 ]]


In [10]:
train_features = train_data.iloc[:,1:]
train_labels = train_data.iloc[:,0]

In [65]:
from sklearn.model_selection import RepeatedKFold
random_state = 12883823
rkf = RepeatedKFold(n_splits=3, n_repeats=5, random_state=random_state)
for train_idx, test_idx in rkf.split(X_train_data):
#     print(train_idx,type(train_idx))
    X_train = X_train_data[train_idx]
    y_train = y_train_data[train_idx]
    X_test = X_train_data[test_idx]
    y_test = y_train_data[test_idx]
    
    X_train = nd.array(X_train,dtype="float32")
    X_test = nd.array(X_test,dtype="float32")
    
    y_train = nd.array(y_train)
    y_test = nd.array(y_test)
    
    train_set= gdata.ArrayDataset(X_train,y_train)
    test_set = gdata.ArrayDataset(X_test,y_test)
    
    batch_size = 10
    train_iter = gdata.DataLoader(train_set,batch_size,shuffle=True)
    test_iter = gdata.DataLoader(test_set,batch_size,shuffle=True)
    
    num_epochs = 5
    gb.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,trainer)
    
commit = nd.array(commit,dtype="float32")

epoch 1, loss 0.3178, train acc 0.865, test acc 0.879
epoch 2, loss 0.2998, train acc 0.867, test acc 0.882
epoch 3, loss 0.2980, train acc 0.873, test acc 0.875
epoch 4, loss 0.2812, train acc 0.873, test acc 0.869
epoch 5, loss 0.2980, train acc 0.873, test acc 0.842
epoch 1, loss 0.3527, train acc 0.850, test acc 0.848
epoch 2, loss 0.3153, train acc 0.871, test acc 0.875
epoch 3, loss 0.3013, train acc 0.878, test acc 0.865
epoch 4, loss 0.2882, train acc 0.877, test acc 0.832
epoch 5, loss 0.2951, train acc 0.876, test acc 0.838
epoch 1, loss 0.3506, train acc 0.852, test acc 0.879
epoch 2, loss 0.2972, train acc 0.876, test acc 0.865
epoch 3, loss 0.2876, train acc 0.879, test acc 0.855
epoch 4, loss 0.2859, train acc 0.871, test acc 0.845
epoch 5, loss 0.2719, train acc 0.878, test acc 0.848
epoch 1, loss 0.3144, train acc 0.875, test acc 0.859
epoch 2, loss 0.2975, train acc 0.878, test acc 0.862
epoch 3, loss 0.2979, train acc 0.883, test acc 0.862
epoch 4, loss 0.2878, train 

In [61]:
net = nn.Sequential()
net.add(nn.Dense(20,activation="relu"),
        nn.Dense(8,activation="relu"),
    nn.Dense(2))

In [62]:
net.initialize(init.Normal(sigma=0.01))
loss = gloss.SoftmaxCrossEntropyLoss()

In [63]:
trainer = gluon.Trainer(net.collect_params(),'adam',{"learning_rate":0.01})

In [66]:
submission = pd.DataFrame({"PassengerId":pd.read_csv("test.csv")["PassengerId"],
                          "Survived":net(commit).argmax(axis=1).asnumpy().astype("int")
                          })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [67]:
submission.to_csv("submission_mxnet_gluon.csv",index=False)